<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватель Факультета Компьютерных Наук НИУ ВШЭ <br> Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center> Занятие 5. Композиции алгоритмов, случайный лес</center>
## <center>Практика. Деревья решений и случайный лес в соревновании Kaggle Inclass по кредитному скорингу. Решение</center>

**[Соревнование](https://inclass.kaggle.com/c/beeline-credit-scoring-competition-2).**

Решается задача кредитного скоринга. 

Признаки клиентов банка:
- Age - возраст (вещественный)
- Income - месячный доход (вещественный)
- BalanceToCreditLimit - отношение баланса на кредитной карте к лимиту по кредиту (вещественный)
- DIR - Debt-to-income Ratio (вещественный)
- NumLoans - число заемов и кредитных линий
- NumRealEstateLoans - число ипотек и заемов, связанных с недвижимостью (натуральное число)
- NumDependents - число членов семьи, которых содержит клиент, исключая самого клиента (натуральное число)
- Num30-59Delinquencies - число просрочек выплат по кредиту от 30 до 59 дней (натуральное число)
- Num60-89Delinquencies - число просрочек выплат по кредиту от 60 до 89 дней (натуральное число)
- Delinquent90 - были ли просрочки выплат по кредиту более 90 дней (бинарный) - имеется только в обучающей выборке

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_auc_score
%pylab inline

**Загружаем данные.**

In [ ]:
train_df = pd.read_csv('../../data/credit_scoring_train.csv', index_col='client_id')
test_df = pd.read_csv('../../data/credit_scoring_test.csv', index_col='client_id')

In [ ]:
y = train_df['Delinquent90']
train_df.drop('Delinquent90', axis=1, inplace=True)

In [ ]:
train_df.head()

**Посмотрим на число пропусков в каждом признаке.**

In [ ]:
train_df.info()

In [ ]:
test_df.info()

**Заменим пропуски медианными значениями.**

In [ ]:
train_df['NumDependents'].fillna(train_df['NumDependents'].median(), inplace=True)
train_df['Income'].fillna(train_df['Income'].median(), inplace=True)
test_df['NumDependents'].fillna(test_df['NumDependents'].median(), inplace=True)
test_df['Income'].fillna(test_df['Income'].median(), inplace=True)

### Дерево решений без настройки параметров

**Обучите дерево решений максимальной глубины 3, используйте параметр random_state=42 для воспроизводимости результатов.**

In [ ]:
first_tree = # Ваш код здесь
first_tree.fit # Ваш код здесь

**Сделайте прогноз для тестовой выборки.**

In [ ]:
first_tree_pred = first_tree # Ваш код здесь

**Запишем прогноз в файл.**

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='Delinquent90', index_label="client_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(75000, 
                                                  predicted_labels.shape[0] + 75000),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
write_to_submission_file(first_tree_pred, '../../output/credit_scoring_first_tree.csv')

**Если предсказывать вероятности дефолта для клиентов тестовой выборки, результат будет намного лучше.**

In [ ]:
first_tree_pred_probs = first_tree.predict_proba(test_df)[:, 1]

In [ ]:
write_to_submission_file # Ваш код здесь

## Дерево решений с настройкой параметров с помощью GridSearch

**Настройте параметры дерева с помощью GridSearhCV, посмотрите на лучшую комбинацию параметров и среднее качество на 5-кратной кросс-валидации. Используйте параметр random_state=42 (для воспроизводимости результатов), не забывайте про распараллеливание (n_jobs=-1).**

In [9]:
tree_params = {'max_depth': list(range(3,8)), 
               'min_samples_leaf': list(range(5,13))}

locally_best_tree = GridSearchCV # Ваш код здесь
locally_best_tree.fit # Ваш код здесь

<unbound method GridSearchCV.fit>

In [ ]:
locally_best_tree.best_params_, round(locally_best_tree.best_score_, 3)

**Сделайте прогноз для тестовой выборки и пошлите решение на Kaggle.**

In [ ]:
tuned_tree_pred_probs = locally_best_tree # Ваш код здесь

In [ ]:
write_to_submission_file # Ваш код здесь

### Случайный лес без настройки параметров

**Обучите случайный лес из деревьев максимальной глубины 3, используйте параметр random_state=42 для воспроизводимости результатов.**

In [ ]:
first_forest = # Ваш код здесь
first_forest.fit # Ваш код здесь

In [ ]:
first_forest_pred = first_forest # Ваш код здесь

**Сделайте прогноз для тестовой выборки и пошлите решение на Kaggle.**

In [ ]:
write_to_submission_file # Ваш код здесь

### Случайный лес c настройкой параметров

**Настройте параметры леса с помощью GridSearh, посмотрите на лучшую комбинацию параметров и среднее качество на 5-кратной кросс-валидации. Используйте параметр random_state=42 (для воспроизводимости результатов), не забывайте про распараллеливание (n_jobs=-1).**

In [ ]:
%%time
forest_params = {'max_depth': list(range(1,11)),
                 'min_samples_leaf': list(range(1,11))}

locally_best_forest = GridSearchCV # Ваш код здесь
locally_best_forest.fit # Ваш код здесь

In [ ]:
locally_best_forest.best_params_, round(locally_best_forest.best_score_, 3)

In [ ]:
tuned_forest_pred = locally_best_forest # Ваш код здесь

In [ ]:
write_to_submission_file # Ваш код здесь

**Посмотрите, как настроенный случайный лес оценивает важность признаков по их влиянию на целевой. Представьте результаты в наглядном виде с помощью DataFrame.**

In [ ]:
pd.DataFrame(locally_best_forest.best_estimator_.feature_importances_ # Ваш код здесь

**Обычно увеличение количества деревьев только улучшает результат. Так что напоследок обучите случайный лес из 300 деревьев с найденными лучшими параметрами. Это может занять несколько минут.**

In [ ]:
%%time
final_forest = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1, # Ваш код здесь
final_forest.fit(train_df, y)
final_forest_pred = final_forest.predict_proba(test_df)[:, 1]
write_to_submission_file(final_forest_pred, '../../output/credit_scoring_final_forest.csv')

**Сделайте посылку на Kaggle.**